<a target="_blank" href="https://colab.research.google.com/github/MarkHmnv/Toxic-Comment-Classifier/blob/main/train.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Upload you kaggle API token(kaggle.json)

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

In [ ]:
# Add kaggle API token
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download the dataset and unzip it
!kaggle datasets download -d fizzbuzz/cleaned-toxic-comments
!unzip cleaned-toxic-comments.zip

In [ ]:
# If you use Google Colab - clone the repository and move model.py to the root folder
!git clone https://github.com/MarkHmnv/Toxic-Comment-Classifier.git
!mv Toxic-Comment-Classifier/model.py /content/

In [ ]:
import pandas as pd
import seaborn as sns
import tensorflow as tf
from model import ToxicCommentClassifier
from sklearn.model_selection import train_test_split

sns.set()

In [ ]:
# Load the data
data = pd.read_csv('train_preprocessed.csv')

In [ ]:
# Split the data into training and validating sets
X = data['comment_text'].values
y = data['toxic']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Create tensorflow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))

train_dataset = train_dataset.batch(32)
valid_dataset = valid_dataset.batch(32)

In [ ]:
# Initialize the model and start the training
model = ToxicCommentClassifier()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

history = model.fit(train_dataset, validation_data=valid_dataset, epochs=5)

In [ ]:
# Plot training and validation accuracy and loss
pd.DataFrame(history.history).plot()

In [ ]:
# You can test the model predictions
sentence = ["It's very nice weather today."]
prediction = model.predict(sentence)
prediction[0]

In [ ]:
# Save the model
model.save('/content')

In [ ]:
# If you use Google Colab you can download the model
def download_model():
    from google.colab import files

    files.download('/content/saved_model.pb')

download_model()